# **One-time Init**

In [ ]:
from os import chdir, environ
environ['TZ'] = 'US/Central'
!nvidia-smi

Sat Jan  1 22:07:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.49       Driver Version: 496.49       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0  On |                  N/A |
|  0%   51C    P5    16W / 170W |   4091MiB / 12288MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/google_drive', force_remount=True)
!unlink /content/drive
!ln -s /content/google_drive/MyDrive/ /content/drive

ModuleNotFoundError: ignored

In [ ]:
# For debug
from os import environ
# environ['CUDA_LAUNCH_BLOCKING'] = '1'

# **Starting Work**

In [1]:
from os import chdir, environ
environ['TZ'] = 'US/Central'
chdir('PointCloud/')
# chdir('/content/drive/PointCloud/')

In [2]:
global logging_init_flag
logging_init_flag = False

def init_logging(OUTPUT):
    global logging_init_flag
    if logging_init_flag:
        return
    logging_init_flag = True

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s:\t%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

    fh = logging.FileHandler(f"{OUTPUT}/training.log")
    fh.setLevel(logging.INFO)
    fh.setFormatter(formatter)

    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)

    logger.addHandler(ch)
    logger.addHandler(fh)



In [3]:
!ls -lt --time-style='+%y-%m-%d %H:%M:%S'
!dir

'ls' is not recognized as an internal or external command,
operable program or batch file.


 Volume in drive C is OS
 Volume Serial Number is 3A4D-1115

 Directory of C:\Users\immor\Colab\PointCloud

02/11/2022  21:41    <DIR>          .
02/12/2022  19:57    <DIR>          ..
10/27/2021  09:09            53,575 build_tree
11/01/2021  22:50             3,795 build_tree.cpp
02/10/2022  23:59            20,765 build_tree.py
11/01/2021  22:50             2,769 build_tree_basic.cpp
11/01/2021  22:50             5,065 build_tree_extra.cpp
10/24/2021  09:45             1,969 build_tree_toy.py
02/11/2022  21:41             8,778 dataset.py
12/03/2021  11:10    <DIR>          datasets
09/23/2021  13:13               208 def.sh
09/24/2021  10:16           579,198 dev.ans
10/27/2021  09:09           350,568 dev.out
02/11/2022  15:03            15,225 encoder.py
10/31/2021  11:26             9,195 model.py
10/17/2021  15:20            12,386 model_arrange.py
11/02/2021  20:28            14,641 model_attention.py
10/23/2021  06:33            14,284 model_carry.py
09/30/2021  21:57        

# **Experiment Init**

In [4]:
import json
import h5py
import torch
import os
from encoder import *
# from encoder import MLP
from res_encoder import ResEncoder
from build_tree import get_directions, init_directions
import logging
from dataset import *

# orig
# prefix = "_orig_pca_symloss_x2"
# no_prealign = True 
# rotate_only = False
# augment = 2
# transform = no_transform 
# use_symmetry_loss = True
# dim = 4096 # 2048
# lowrk = 128
# dim_layer0 = 32 # 16
# dim_repeat_cut = 5
# train_augment = False
# permute_augment = False #False
# model_size = 2 ** 11
# num_res_layer = 0
# carry_dim = 64
# upload_remove_equal = True
# use_small_point2embed = True
# model_name = 'Encoder'

# affined
prefix = "_affine_pca_nosample_fixed_x20"
no_prealign = False 
rotate_only = False
augment = 16
transform = affine_transform 
use_symmetry_loss = False
dim = 4096 # 2048
lowrk = 999999999
dim_layer0 = 32 # 16
dim_repeat_cut = 5 # 5
train_augment = False
permute_augment = True
model_size = 2 ** 11
num_layers = 1
num_res_layers = 1
carry_dim = dim_layer0 * 8
upload_remove_equal = False # False
use_small_point2embed = False
model_name = 'ResEncoder' # 'EncoderKdtAlign'

# affined noprealign
# prefix = "_affine_pca_nosample_noprealign_x16"
# no_prealign = True #False 
# rotate_only = False
# augment = 16
# transform = affine_transform 
# use_symmetry_loss = False
# dim = 4096 # 2048
# lowrk = 999999999
# dim_layer0 = 32 # 16
# dim_repeat_cut = 5 # 5
# train_augment = False
# permute_augment = True
# model_size = 2 ** 11
# num_layers = 1
# num_res_layers = 1
# carry_dim = dim_layer0 * 8
# upload_remove_equal = False # False
# use_small_point2embed = False
# model_name = 'ResEncoder' # 'EncoderKdtAlign'

OUTPUT = 'scratch_extra'

sample_layers = 50 # 2
channel = 1


sample_child_first = False # True in l7s1
num_classes = 40
DATASET = './datasets/ModelNet40'
chaos_limit = 0

init_logging(OUTPUT)
logging.info(f"prefix = {prefix}")
_ = init_directions(chaos_limit, calc_dmap=False)


2022-02-13 22:30:19 - INFO:	prefix = _affine_pca_nosample_fixed_x20
2022-02-13 22:30:19 - INFO:	init_directions: # = 3
2022-02-13 22:30:19 - DEBUG:	0: 0.000000 1.000000 0.000000 otho = {1, 2}
2022-02-13 22:30:19 - DEBUG:	1: 1.000000 0.000000 0.000000 otho = {0, 2}
2022-02-13 22:30:19 - DEBUG:	2: 0.000000 0.000000 1.000000 otho = {0, 1}
2022-02-13 22:30:19 - INFO:	basic # = 3
2022-02-13 22:30:19 - INFO:	transforms # = 48


In [5]:
torch.manual_seed(674433238)
has_align_feature = False
if model_name == 'Encoder':
    model = Encoder(model_size, sample_layers, dim, OUTPUT, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'ResEncoder':
    has_align_feature = True
    model = ResEncoder(num_res_layers, carry_dim, model_size, sample_layers, dim, OUTPUT, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'EncoderKdtAlign':
    has_align_feature = True
    model = EncoderKdtAlign(model_size, sample_layers, dim, OUTPUT, num_layers=num_layers, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
elif model_name == 'EncoderRec':
    model = EncoderRec(model_size, sample_layers, dim, OUTPUT, num_layers=2, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut, use_symmetry_loss=use_symmetry_loss).cuda()
else:
    assert False, f"Unsupported {model_name}"
model.dim

2022-02-13 22:30:19 - INFO:	self.N = 2048
2022-02-13 22:30:19 - INFO:	self.num_layers = 12
2022-02-13 22:30:19 - INFO:	self.sample_layers = 50
2022-02-13 22:30:19 - INFO:	layer 0 (leaf) # = 2048 odim = 32
2022-02-13 22:30:21 - INFO:	layer 1 (unsampled) # = 1024 odim = 64
2022-02-13 22:30:21 - INFO:	layer 2 (unsampled) # = 512 odim = 128
2022-02-13 22:30:21 - INFO:	layer 3 (unsampled) # = 256 odim = 256
2022-02-13 22:30:21 - INFO:	layer 4 (unsampled) # = 128 odim = 512
2022-02-13 22:30:21 - INFO:	layer 5 (unsampled) # = 64 odim = 512
2022-02-13 22:30:21 - INFO:	layer 6 (unsampled) # = 32 odim = 1024
2022-02-13 22:30:21 - INFO:	layer 7 (unsampled) # = 16 odim = 1024
2022-02-13 22:30:21 - INFO:	layer 8 (unsampled) # = 8 odim = 2048
2022-02-13 22:30:21 - INFO:	layer 9 (unsampled) # = 4 odim = 2048
2022-02-13 22:30:21 - INFO:	layer 10 (unsampled) # = 2 odim = 4096
2022-02-13 22:30:21 - INFO:	layer 11 (unsampled) # = 1 odim = 4096
2022-02-13 22:30:21 - INFO:	layer 0 feature_dim = 4096 globe_

4096

In [6]:
def hack(model):
    if upload_remove_equal:
        for i, layer in enumerate(model.layers):
            if layer.idim == layer.odim:
                layer.upload = torch.nn.Identity().cuda()
                print(f"#{i}: Set upload to identity.")
    if lowrk < dim:
        from encoder import convert_lowrk
        # pts_align = model.layers[0].pts_align
        # for i in range(6):
        #     pts_align.fc[i] = convert_lowrk(pts_align.fc[i], lowrk).cuda()
        # for l in model.layers[0].mlp.layers:
        #     l.linear = convert_lowrk(l.linear, lowrk).cuda()
        for i, layer in enumerate(model.layers):
            if layer.layer_type != 'leaf' and isinstance(layer.upload, MLP):
                for l in layer.upload.layers:
                    l.linear = convert_lowrk(l.linear, lowrk).cuda()
                print(f"#{i}: Converted to lowrk.")
    if use_small_point2embed:
        l = model.layers[0]
        l.mlp = MLP([l.idim, l.odim * 4, l.odim], init=0.25).cuda()
        print(f"Reset middle layer of point2embed")

def hack_seg(model):
    hack(model.encoder)

if isinstance(model, Encoder):
    hack(model)
elif isinstance(model, EncoderKdtAlign):
    hack(model.encoder)
    upload_remove_equal = True
    use_small_point2embed = True

    def hack_rec(model):
        if isinstance(model, Encoder):
            hack(model)
        else:
            hack(model.encoder)
            hack_rec(model.align.encoder)

    hack_rec(model.align.encoder)
elif isinstance(model, EncoderRec):
    hack(model.encoder)
elif isinstance(model, ResEncoder):
    print("feed:")
    hack(model.feed)
    # upload_remove_equal = True
    # use_small_point2embed = True
    print("layers:")
    for seg in model.layers:
        hack_seg(seg)
else:
    print("Unsupported")
    # assert False, "Unsupported"

feed:
layers:


In [7]:
model

ResEncoder(
  (layers): ModuleList(
    (0): Segment(
      (encoder): Encoder(
        (layers): ModuleList(
          (0): EncoderLayer(
            (pts_align): Alignment(
              (conv1): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
              (conv2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
              (conv3): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
              (fc1): Linear(in_features=1024, out_features=512, bias=True)
              (fc2): Linear(in_features=512, out_features=256, bias=True)
              (fc3): Linear(in_features=256, out_features=9, bias=True)
              (relu): ReLU()
              (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (bn4): BatchNorm1d(512, eps=1e-05, momentu

# **Debug**

In [ ]:
assert False, "debug part cannot be run through"

AssertionError: ignored

In [ ]:
pts = h5py.File(f'{DATASET}/train0.h5')['data'][0]
pts = torch.tensor(pts)
output = None

<ipython-input-8-17a3ae9dea0c>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  pts = h5py.File(f'{DATASET}/train0.h5')['data'][0]


In [ ]:
pts = transform(pts)

In [ ]:
tree = model.tree
pts, output, extra = model.tree.arrange(pts, rotate=not no_prealign, extra=False, pca=True,
                                          rotate_only=rotate_only, debug=False)

In [ ]:
%matplotlib inline
def plot(points, color_layers=2, output=None):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.gridspec as gridspec
    import numpy as np

    points = points.cpu().numpy()
    x, y, z = points[:,0], points[:,1], points[:,2]
    x -= x.mean()
    y -= y.mean()
    z -= z.mean()

    color = np.array([0 for _ in range(x.shape[0])])
    
    def mark_color(z, last_ind=None):
        n = z.shape[0]
        ind = torch.tensor(z).sort()[1].numpy()
        indl, indr = ind[: n >> 1], ind[n >> 1 : ]
        if last_ind is not None:
            indl, indr = last_ind[indl], last_ind[indr]
        color[indr] += 1
        return indl, indr

    if output is not None:

        color = torch.tensor([0]).cuda()
        # print(tree.layer_size)
        for i, (layer, next_layer_size) in enumerate(zip(reversed(model.layers[1:]), tree.layer_size[1:])):
            next_color = torch.zeros(next_layer_size).long().cuda()
            if i < color_layers:
                color = color << 1
                right_add = 1
            else:
                right_add = 0
            assert layer.child_l.max() < next_layer_size
            assert layer.child_r.max() < next_layer_size
            next_color[layer.child_l] = color
            next_color[layer.child_r] = color + right_add
            color = next_color

        arrange = output[0]
        n = points.shape[0]
        color = np.array([color[arrange == i].max().item() for i in range(n)])

    elif color_layers >= 1:
        l, r = mark_color(z)
        if color_layers >= 2:
            color *= 2
            ll, lr = mark_color(y[l], l)
            rl, rr = mark_color(y[r], r)
            if color_layers >= 3:
                color *= 2
                for p in [ll, lr, rl, rr]:
                    mark_color(x[p], p)

    colormap = np.array(['red', 'blue', 'green', 'yellow', 'grey', 'orange', 'purple', 'cyan'])

    fig = plt.figure(dpi=80)
    gs = gridspec.GridSpec(nrows=2, ncols=4, left=0.1, right=2.5, wspace=0.05, hspace=0.05, bottom=0.1, top=1.3)
    for i in range(8):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        x, y, z = points[:,0], points[:,1], points[:,2]
        labx, laby, labz = 'x', 'y', 'z'
        if (i & 4) != 0:    x = -x; labx = '-x'
        if (i & 2) != 0:    y = -y; laby = '-y'
        if (i & 1) != 0:    z = -z; labz = '-z'

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[color], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel(labx)
        ax.set_ylabel(laby)
        ax.set_zlabel(labz) 
    plt.show()

    fig = plt.figure(dpi=80)
    ncolor = color.max() + 1
    nrows = max(1, ncolor // 4)
    ncols = min(ncolor, 4)
    gs = gridspec.GridSpec(nrows=nrows, ncols=ncols, left=0.1, right=0.1 + 0.6 * ncols, wspace=0.05, hspace=0.05, bottom=0.1, top=0.1 + 0.6 * nrows)
    for i in range(ncolor):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        p = points[color == i]
        x, y, z = p[:,0], p[:,1], p[:,2]

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[i], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z') 
    plt.show()

In [ ]:
plot(pts, 3, output=output)

In [ ]:
model.align.dim

In [ ]:
batch_size = 64
model.train()
for _ in range(8):
    _pts = pts.unsqueeze(0).expand(batch_size, *pts.shape)
    _output = list(map(lambda x : x.unsqueeze(0).expand(batch_size, *x.shape), output))
    _extra = extra.unsqueeze(0).expand(batch_size, *extra.shape)
    feature = model(_pts, _output, _extra, perm=0)
    print(feature.shape)
    (feature.sum()).backward()

torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])
torch.Size([64, 4096])


In [ ]:
plot_tri(pts, tri_ind)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def plot_tri(points, tri):
    points = points.numpy()
    tri = tri.numpy()

    fig = plt.figure(dpi=200)
    ax = fig.add_subplot(111, projection='3d')
    
    edges = collect_edges(tri)
    x = np.array([])
    y = np.array([])
    z = np.array([])
    for (i,j) in edges:
        x = np.append(x, [points[i, 0], points[j, 0], np.nan])      
        y = np.append(y, [points[i, 1], points[j, 1], np.nan])      
        z = np.append(z, [points[i, 2], points[j, 2], np.nan])
    ax.plot3D(x, y, z, color='g', lw='0.1')

    ax.scatter(points[:,0], points[:,1], points[:,2], color='b', s=1)


def collect_edges(tri):
    edges = set()

    def sorted_tuple(a,b):
        return (a,b) if a < b else (b,a)
    # Add edges of tetrahedron (sorted so we don't add an edge twice, even if it comes in reverse order).
    for (i0, i1, i2, i3) in tri:
        edges.add(sorted_tuple(i0,i1))
        edges.add(sorted_tuple(i0,i2))
        edges.add(sorted_tuple(i0,i3))
        edges.add(sorted_tuple(i1,i2))
        edges.add(sorted_tuple(i1,i3))
        edges.add(sorted_tuple(i2,i3))
    return edges

In [ ]:
pts = train_data[7][0]
output = None

In [ ]:
for i in range(50):
    output, arrange = tree.arrange(pts, basic=basic, debug=False, device='cpu')

In [ ]:
feature.shape

In [ ]:
for i in range(10):
    loss = model(pts.unsqueeze(0), list(map(lambda x : x.unsqueeze(0), output))).view(-1).sum()
    loss.backward()

In [ ]:
model

EncoderKdtAlign(
  (align): AlignWithEncoder(
    (encoder): EncoderKdtAlign(
      (align): AlignWithEncoder(
        (encoder): Encoder(
          (layers): ModuleList(
            (0): EncoderLayer(
              (pts_align): Alignment(
                (conv1): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
                (conv2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
                (conv3): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
                (fc1): Linear(in_features=1024, out_features=512, bias=True)
                (fc2): Linear(in_features=512, out_features=256, bias=True)
                (fc3): Linear(in_features=256, out_features=9, bias=True)
                (relu): ReLU()
                (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
for name, param in model.named_parameters():
    print(name, param.size())

align.encoder.align.encoder.layers.0.pts_align.conv1.weight torch.Size([128, 3, 1])
align.encoder.align.encoder.layers.0.pts_align.conv1.bias torch.Size([128])
align.encoder.align.encoder.layers.0.pts_align.conv2.weight torch.Size([256, 128, 1])
align.encoder.align.encoder.layers.0.pts_align.conv2.bias torch.Size([256])
align.encoder.align.encoder.layers.0.pts_align.conv3.weight torch.Size([1024, 256, 1])
align.encoder.align.encoder.layers.0.pts_align.conv3.bias torch.Size([1024])
align.encoder.align.encoder.layers.0.pts_align.fc1.weight torch.Size([512, 1024])
align.encoder.align.encoder.layers.0.pts_align.fc1.bias torch.Size([512])
align.encoder.align.encoder.layers.0.pts_align.fc2.weight torch.Size([256, 512])
align.encoder.align.encoder.layers.0.pts_align.fc2.bias torch.Size([256])
align.encoder.align.encoder.layers.0.pts_align.fc3.weight torch.Size([9, 256])
align.encoder.align.encoder.layers.0.pts_align.fc3.bias torch.Size([9])
align.encoder.align.encoder.layers.0.pts_align.bn1.w

In [ ]:
for name, param in model.named_parameters():
    if 'activate' in name:
        print(name, "\t", param.item())

In [ ]:
!cp /tmp/cppinput.txt scratch/

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!tail -n 5 /tmp/cppoutput.txt

'tail' is not recognized as an internal or external command,
operable program or batch file.


# **Data**

In [8]:
from dataset import *
import numpy as np

In [9]:
if False:
    from random import shuffle, seed
    raw_data = torch.load(f'{OUTPUT}/train_data{prefix}.pth')
    n = len(raw_data) // data_augments
    perm = list(range(n))
    seed(674433238)
    shuffle(perm)
    train_perm = perm[: -n//8]
    valid_perm = perm[-n//8: ]
    torch.save([train_perm, valid_perm], f'{OUTPUT}/train_split{prefix}.pth')

In [10]:
make = make_data_default
if rotate_only:
    make = make_data_rotate_only
if no_prealign:
    make = make_data_no_prealign

In [11]:
n_train = 5
clouds = []
labels = []
for i in range(n_train):
    data_file = h5py.File(f'{DATASET}/train{i}.h5')
    clouds.append(torch.tensor(np.array(data_file['data'])))
    labels.append(torch.tensor(np.array(data_file['label'])))

clouds = torch.cat(clouds, dim=0)
labels = torch.cat(labels, dim=0)

try:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split{prefix}.pth')
except:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split.pth')

from math import ceil
train_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=augment, transform=transform, make=make, subset=train_perm) #, extra_labels=clouds)
valid_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=max(1, (1 + int(ceil(augment/4) + 0.5) if augment <= 4 else augment // 4)), transform=transform, make=make, subset=valid_perm)

<ipython-input-11-8387f0cba51a>:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data_file = h5py.File(f'{DATASET}/train{i}.h5')


In [12]:
n_test = 2
clouds = []
labels = []
for i in range(n_test):
    data_file = h5py.File(f'{DATASET}/test{i}.h5')
    clouds.append(torch.tensor(np.array(data_file['data'])))
    labels.append(torch.tensor(np.array(data_file['label'])))

clouds = torch.cat(clouds, dim=0)
labels = torch.cat(labels, dim=0)

test_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=1, make=make, transform=transform)

<ipython-input-12-936995772004>:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data_file = h5py.File(f'{DATASET}/test{i}.h5')


In [13]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(137792, 4924, 2468)

In [14]:
target = [] # ['train', 'valid', 'test']

import gc
gc.collect()

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    from tqdm import tqdm

    if name not in target:
        continue

    num_workers = 16
    batch_size = 8

    data_init = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=placeholder, pin_memory=False, drop_last=False)
    logging.info(f"Init {name}")

    mem = []

    def save():
        torch.save(mem, f'{OUTPUT}/{name}_data{prefix}.pth')
        # logging.info("Saved")

    for i, data in enumerate(tqdm(data_init)):
        mem += data
        # logging.debug(f"Init {name}: {i+1}/{len(data_init)}")

        if (i + 1) % (400 * max(1, augment//4)) == 0:
            save()
    
    save()
    logging.info("Done")

    del mem
    gc.collect()


# **Pretrain**

In [ ]:
assert False, "not implemented"

AssertionError: ignored

In [ ]:
class FeatureMap(torch.nn.Module):
    def __init__(self, dim):
        super(FeatureMap, self).__init__()
        self.relu = torch.nn.ReLU()

        self.linear1 = torch.nn.Linear(dim, dim // 2)
        self.bn1 = torch.nn.BatchNorm1d(dim // 2)

        self.linear2 = torch.nn.Linear(dim // 2, dim // 4)
        self.bn2 = torch.nn.BatchNorm1d(dim // 4)

        self.linear3 = torch.nn.Linear(dim // 4, dim // 8)
        self.bn3 = torch.nn.BatchNorm1d(dim // 8)
        self.dropout = torch.nn.Dropout(0.7)

    def forward(self, ans):
        ans = self.linear1(ans)
        ans = self.bn1(ans)
        ans = self.relu(ans)

        ans = self.linear2(ans)
        ans = self.dropout(ans)
        ans = self.bn2(ans)
        ans = self.relu(ans)
        
        ans = self.linear3(ans)
        ans = self.bn3(ans)
        return ans

torch.manual_seed(674433238)
feature_map = FeatureMap(dim).cuda()

In [ ]:
try:
    raw_data
except:
    raw_data = torch.load(f'{OUTPUT}/train_data{prefix}.pth')
try:
    test_data
except:
    test_data = torch.load(f'{OUTPUT}/test_data{prefix}.pth')
try:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split{prefix}.pth')
except:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split.pth')

n = len(raw_data)

train_data = [raw_data[i * data_augments + j] for i in train_perm for j in range(data_augments)]
valid_data = [raw_data[i * data_augments + j] for i in valid_perm for j in range(data_augments)]

classes = [[] for i in range(num_classes)]
for (pts, output, extra, label) in train_data:
    classes[label.item()].append((pts, output, extra, label))

valid_classes = [[] for i in range(num_classes)]
for (pts, output, extra, label) in valid_data:
    valid_classes[label.item()].append((pts, output, extra, label))

from random import shuffle, seed
seed(674433238)
shuffle(train_data)
shuffle(valid_data)
shuffle(test_data)

In [ ]:
def make_batch(batches, eval=False):
    from random import randint
    layers = 0
    points = []
    inputs = None
    labels = []
    extras = []
    for pts, output, extra, label in batches:
        # Simple data augment
        # sgn = torch.tensor(-1).pow(torch.randint(low=0, high=2, size=[extra.shape[-1]])).cuda()
        sgn = (-1) ** randint(0, 1)
        
        points.append(pts.cuda())
        labels.append(label.cuda())
        extras.append(extra.cuda() * (1 if eval else sgn))
        if inputs is None:
            layers = len(output)
            inputs = [[] for _ in output]

        for line, out in zip(inputs, output):
            line.append(out.cuda())

    points = torch.stack(points, dim=0).float().cuda()
    extras = torch.stack(extras, dim=0).float().cuda()
    for i, line in enumerate(inputs):
        inputs[i] = torch.stack(line, dim=0).long().cuda()
    labels = torch.stack(labels, dim=0).long().cuda()
    
    return (points, inputs, extras), labels

In [ ]:
from random import choice
num_samples = 2
temperature = 0.07

class_size = torch.tensor([len(c) for c in classes]).float().cuda()
contrast_crit = torch.nn.CrossEntropyLoss(reduce=None)

def contrast_batch(used_classes=classes):
    batches = []
    for _ in range(num_samples):
        for c in used_classes:
            batches.append(choice(c))
    return make_batch(batches)

def contrast_loss(features, labels):
    num_data = num_classes * num_samples

    ratio = class_size[labels]
    ratio /= ratio.sum()
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1))

    features = torch.nn.functional.normalize(features, dim=1)

    sim = features.matmul(features.T)

    mask = torch.eye(num_data).bool().cuda()
    labels = labels[~mask].view(num_data, -1)
    sim = sim[~mask].view(num_data, -1)

    pos = sim[labels].view(num_data, -1)
    neg = sim[~labels].view(num_data, -1)

    logits = torch.cat([pos, neg], dim=1) / temperature

    correct = ((logits.argmax(dim=-1) == 0) * ratio).sum().item() * num_data

    loss = (contrast_crit(logits, torch.zeros(num_data).long().cuda()) * ratio).sum()

    return loss, correct

def valid_eval(classes=valid_classes):
    model.eval()
    feature_map.eval()

    with torch.no_grad():

        cum_loss = 0
        cum_correct = 0
        cum_1 = 0

        for _ in range(8):
            input, labels = contrast_batch(classes)
            features = feature_map(model(*input))
            l, c = contrast_loss(features, labels)

            cum_loss += l.item()
            cum_correct += c / (num_classes * num_samples)
            cum_1 += 1

        valid_str = "valid = %.4lf (%.6lf)" % (cum_loss / cum_1, cum_correct / cum_1)

    model.train()
    feature_map.train()

    return valid_str


In [ ]:
from random import choice, randint, seed
import build_tree
import torch
from math import sin, acos

logging.info(f"train_data # = {len(train_data)}")


model.train()
feature_map.train()
activate = lambda x : x

pi = acos(-1)
cum_loss = 0

batch_scale = 1 # 1 if basic else 1
epoch_scale = 4 # 4 if basic else 2
num_trans = 1

print_epoch = 50
valid_epoch = 1
epoch_since = 0
save_epoch = 500
cut_epoch = 10000000000
correct = 0
total = 0

debug_print = 5

def erate_value(epoch):
    N = 20000
    return torch.tensor(sin(min(1, epoch / N) * pi / 2)).cuda()

threshold = -1.0

num_epoch = 100000

crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(list(model.parameters()) + list(feature_map.parameters()), lr=1e-4)
# sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=num_epoch / 5, eta_min=1e-5)
sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.9999)

def get_trans(n=3):
    if n == 0:  return [None]
    return [randint(0, len(build_tree.transforms) - 1) for _ in range(n)]

In [ ]:
seed(674433238)
torch.manual_seed(674433238)

best_vres = -1.0

for epoch in range(1, num_epoch + 1):
    for _ in range(epoch_scale):
        loss = 0.
        for _ in range(batch_scale):
            input, labels = contrast_batch()
            for iperm in get_trans(num_trans):

                features = feature_map(model(*input, perm=iperm))
                l, c = contrast_loss(features, labels)
                loss += l

                with torch.no_grad():
                    correct += c
                    total += labels.shape[0]
                    epoch_since += 1
        
        # assert loss.isnan().sum() == 0
        cum_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    sch.step()
        
    if epoch <= debug_print or epoch % print_epoch == 0:
        valid_str = ""
        func = logging.debug
        func(f"train #{epoch} lr = {'%.2e' % sch.get_last_lr()[0]} train = {'%.4lf (%.6lf)' % (cum_loss / epoch_since, correct / total)} {valid_eval()}")
        epoch_since = cum_loss = correct = total = 0

    if epoch % save_epoch == 0 or epoch == debug_print:
        torch.save({
            'encoder': model.state_dict(),
            'feature_map': feature_map.state_dict(),
        }, f"{OUTPUT}/pretrained_{epoch}.pth")
        logging.info(f"Saved")

# **Train**

In [ ]:
from dataset import make_batch_train, make_batch_eval

global batch_size
num_workers = 0
batch_size = 32 #64

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    print(f"Loading {name}")
    dataset.mem = torch.load(f'{OUTPUT}/{name}_data{prefix}.pth')
    assert len(dataset.mem) == len(dataset)

mbtrain = make_batch_generator(dropout=0.5, augment=train_augment) if 'orig' in prefix else make_batch_train

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=mbtrain, pin_memory=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)

Loading train


In [ ]:
torch.manual_seed(674433238)

class Classifier(torch.nn.Module):
    def __init__(self, idim, odim, dropout=0):
        super(Classifier, self).__init__()
        self.relu = torch.nn.ReLU()

        self.linear1 = make_lowrk(idim, min(idim // 2, 1024), lowrk)
        self.bn1 = torch.nn.BatchNorm1d(min(idim // 2, 1024))

        # self.linear2 = make_lowrk(idim // 2, idim // 4, lowrk)
        # self.bn2 = torch.nn.BatchNorm1d(idim // 4)
        self.linear2 = torch.nn.Identity()
        self.bn2 = torch.nn.Identity()
        self.dropout = torch.nn.Dropout(dropout)

        # self.linear3 = make_lowrk(idim // 4, odim, lowrk)
        self.linear3 = make_lowrk(min(idim // 2, 1024), odim, lowrk)

    def forward(self, ans):
        ans = self.linear1(ans)
        ans = self.bn1(ans)
        ans = self.relu(ans)

        ans = self.linear2(ans)
        ans = self.bn2(ans)
        ans = self.dropout(ans)
        ans = self.relu(ans)
        
        ans = self.linear3(ans)
        return ans

if 'orig' in prefix:
    from encoder import make_lowrk
    linear = Classifier(model.dim, num_classes, 1 - 1/8).cuda()
    linear_align = torch.nn.Identity().cuda()
else:
    linear = MLP([model.dim, min(1024, model.dim // 2), min(512, model.dim // 4), num_classes], last_bn=False).cuda()
    linear_align = MLP([model.align_dim, min(1024, model.align_dim // 2), min(512, model.align_dim // 4), num_classes], last_bn=False).cuda() if has_align_feature else torch.nn.Identity().cuda()
    # linear = Classifier(model.dim, num_classes, 1 - 1/4).cuda()
    # linear_align =  Classifier(model.dim, num_classes, 1 - 1/4).cuda()

print(linear)
print(linear_align)

In [ ]:
def evaluate(model, linear, loader, noprint=False, perms=[None]):
    correct = 0
    total = 0

    if not noprint:
        logging.info(f"loader # = {len(loader)}")

    print_epoch = 1

    model.eval()
    linear.eval()
    activate = lambda x : x

    for epoch, (input, label) in enumerate(loader):
        with torch.no_grad():
            label = label.squeeze(-1).cuda()
            for perm in perms:
                result = activate(linear(model(*input, perm=perm).contiguous().view(label.size(0), -1)))

                correct += (result.argmax(dim=-1) == label.cuda()).sum().item()
                total += label.size()[0]

        if not noprint:
            if (epoch // batch_size + 1) % print_epoch == 0:
                logging.debug(f"test #{epoch} correct = {'%.6lf' % (correct / total)}")

    if not noprint:
        logging.info(f"Done: score = {'%.8lf' % (correct / total)}")

    model.train()
    linear.train()  
    
    return correct / total

In [ ]:
from random import choice, randint
import build_tree
import torch

def get_trans(n=3):
    if n == 0:  return [None]
    return [randint(0, len(build_tree.transforms) - 1) for _ in range(n)]

def save(epoch):
    torch.save({
        'encoder': model.state_dict(),
        'linear': linear.state_dict(),
        # 'opt': opt.state_dict(),
        # 'sch': sch.state_dict(),
        # 'best_vres': best_vres,
    }, f"{OUTPUT}/trained_{epoch}.pth")

In [ ]:
# if 'orig' in prefix:
#     ckpt = torch.load(f"{OUTPUT}/pretrained_affine_pca_nosample_x16_20211207.pth")
#     model.load_state_dict(ckpt['encoder'])
#     linear.load_state_dict(ckpt['linear'])
#     print("Pretrain loaded")

In [ ]:
global current_epoch
global best_vres
current_epoch = 0
best_vres = -1.0

def train(more_epoch=100000, valid_result_threshold=1.0):
    global current_epoch
    global best_vres
    global batch_size

    logging.info(f"train epoch = {current_epoch + 1} ~ {more_epoch} threshold = {valid_result_threshold}")

    model.train()
    linear.train()
    activate = lambda x : x

    cum_loss = 0
    cum_loss_align = 0

    batch_scale = 1 # 1
    epoch_scale = 8 # 8
    num_trans = 1 if permute_augment else 0


    print_epoch = 10
    valid_epoch = 10
    epoch_since = 0
    save_epoch = 100
    cut_epoch = 10000000000
    correct = 0
    total = 0

    temperature = 1
    threshold = -1.0

    crit = torch.nn.CrossEntropyLoss()
    crit_align = torch.nn.CrossEntropyLoss()
    opt = torch.optim.Adam(list(model.parameters()) + list(linear.parameters()) + list(linear_align.parameters()), lr=1e-4)
    sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.9999)
    save(current_epoch)

    for epoch in range(current_epoch + 1, current_epoch + more_epoch + 1):

        current_epoch = epoch

        for _ in range(epoch_scale):
            loss = torch.tensor(0.).cuda()
            loss_align = torch.tensor(0.).cuda()
            for _ in range(batch_scale):
                input, label = next(iter(train_loader))
                label = label.squeeze(-1).cuda()
                for iperm in get_trans(num_trans):
                    
                    features = model(*input, perm=iperm).reshape(batch_size, -1)
                    logits = linear(features)
                    loss += crit(logits / temperature, label.cuda())
                    if has_align_feature:
                        logits_align = linear_align(model.align_feature)
                        loss_align += crit_align(logits_align / temperature, label.cuda())

                    with torch.no_grad():
                        correct += (logits.argmax(dim=-1) == label.cuda()).sum().item()
                        total += label.size()[0]
                        epoch_since += 1
            
            # assert loss.isnan().sum() == 0
            cum_loss += loss.item()
            cum_loss_align += loss_align.item()
            opt.zero_grad()
            # loss.backward()
            if 'orig' in prefix:
                # (loss / loss.clamp(min=1e-6).item()).backward()
                all = (loss / loss.clamp(min=1e-6).item()) + (loss_align / loss_align.clamp(min=1e-6).item())
                all.backward()
            else:
                all = loss + loss_align
                all.backward()
            opt.step()
            
        sch.step()

        if cum_loss / epoch_since < threshold:
            epoch_scale, batch_scale = batch_scale, epoch_scale
            
            logging.info("Threshold Reached")
            threshold = -1e10
            
        if epoch <= 5 or epoch % print_epoch == 0:
            valid_str = ""
            func = logging.debug

            stop_training = False
            if epoch % valid_epoch == 0:
                vres = evaluate(model, linear, valid_loader, noprint=True)
                valid_str = f"valid = {'%.6lf' % vres}"
                stop_training = (vres >= valid_result_threshold)
                if vres > best_vres:
                    best_vres = vres
                    torch.save({
                        'encoder': model.state_dict(),
                        'linear': linear.state_dict()
                    }, f"{OUTPUT}/trained_best{prefix}.pth")
                    valid_str += " updated"

                func = logging.info
            func(f"train #{epoch} lr = {'%.2e' % sch.get_last_lr()[0]} loss = {'%.6lf / %.6lf' % (cum_loss / epoch_since, cum_loss_align / epoch_since)} train = {'%.6lf' % (correct / total)} {valid_str}")
            epoch_since = cum_loss = cum_loss_align = correct = total = 0

            if stop_training:
                break


        if epoch % save_epoch == 0:
            save(epoch)
            tres = evaluate(model, linear, test_loader, noprint=True)
            logging.info(f"Saved test = {'%.6lf' % tres}")

        if epoch % cut_epoch == 0:
            if batch_size > 8:
                batch_size //= 2
                epoch_scale *= 2

            logging.info(f"Cut batch_size = {batch_size} epoch_scale = {epoch_scale}")

In [ ]:
# prealign & fixed
train()

# **Train Result Archive**

In [ ]:
assert False, "protected"

In [ ]:
# affined, res encoder, batchsize32
train()

2022-01-23 11:00:13 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-01-23 11:00:20 - DEBUG:	train #1 lr = 1.00e-04 loss = 3.638879 / 3.547632 train = 0.105469 
2022-01-23 11:00:22 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.376059 / 3.340085 train = 0.167969 
2022-01-23 11:00:24 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.197898 / 3.149245 train = 0.160156 
2022-01-23 11:00:26 - DEBUG:	train #4 lr = 1.00e-04 loss = 3.174321 / 3.127006 train = 0.140625 
2022-01-23 11:00:28 - DEBUG:	train #5 lr = 1.00e-04 loss = 2.946732 / 2.891525 train = 0.222656 
2022-01-23 11:00:49 - INFO:	train #10 lr = 9.99e-05 loss = 2.885735 / 2.863505 train = 0.230469 valid = 0.216491 updated
2022-01-23 11:01:20 - INFO:	train #20 lr = 9.98e-05 loss = 2.693135 / 2.656735 train = 0.261328 valid = 0.265638 updated
2022-01-23 11:01:51 - INFO:	train #30 lr = 9.97e-05 loss = 2.581437 / 2.526844 train = 0.291406 valid = 0.266450 updated
2022-01-23 11:02:23 - INFO:	train #40 lr = 9.96e-05 loss = 2.435853 / 2.351837 trai

KeyboardInterrupt: ignored

In [ ]:
# noprealign
train()

2022-02-12 04:13:04 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-02-12 04:13:13 - DEBUG:	train #1 lr = 1.00e-04 loss = 3.789034 / 3.775991 train = 0.050781 
2022-02-12 04:13:15 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.796719 / 3.795945 train = 0.078125 
2022-02-12 04:13:17 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.752708 / 3.808785 train = 0.054688 
2022-02-12 04:13:19 - DEBUG:	train #4 lr = 1.00e-04 loss = 3.663532 / 3.629048 train = 0.078125 
2022-02-12 04:13:21 - DEBUG:	train #5 lr = 1.00e-04 loss = 3.596512 / 3.605820 train = 0.093750 
2022-02-12 04:13:41 - INFO:	train #10 lr = 9.99e-05 loss = 3.480047 / 3.440394 train = 0.109375 valid = 0.099716 updated
2022-02-12 04:14:10 - INFO:	train #20 lr = 9.98e-05 loss = 3.335137 / 3.285247 train = 0.137891 valid = 0.115760 updated
2022-02-12 04:14:40 - INFO:	train #30 lr = 9.97e-05 loss = 3.195616 / 3.170455 train = 0.154297 valid = 0.156377 updated
2022-02-12 04:15:10 - INFO:	train #40 lr = 9.96e-05 loss = 3.052579 / 3.013393 trai

RuntimeError: ignored

In [ ]:
# affine, encoder with kdt align
train()

2022-01-10 12:35:13 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-01-10 12:35:18 - DEBUG:	train #1 lr = 1.00e-04 loss = 3.627517 / 3.607339 train = 0.068359 
2022-01-10 12:35:19 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.420765 / 3.286781 train = 0.117188 
2022-01-10 12:35:21 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.233898 / 3.089891 train = 0.167969 
2022-01-10 12:35:23 - DEBUG:	train #4 lr = 1.00e-04 loss = 3.167123 / 2.987689 train = 0.166016 
2022-01-10 12:35:24 - DEBUG:	train #5 lr = 1.00e-04 loss = 3.053328 / 2.901727 train = 0.199219 
2022-01-10 12:35:37 - INFO:	train #10 lr = 9.99e-05 loss = 2.845225 / 2.751113 train = 0.246484 valid = 0.196791 updated
2022-01-10 12:35:58 - INFO:	train #20 lr = 9.98e-05 loss = 2.551241 / 2.570152 train = 0.316992 valid = 0.271324 updated
2022-01-10 12:36:19 - INFO:	train #30 lr = 9.97e-05 loss = 2.272364 / 2.370227 train = 0.384375 valid = 0.374898 updated
2022-01-10 12:36:41 - INFO:	train #40 lr = 9.96e-05 loss = 2.090628 / 2.175656 trai

KeyboardInterrupt: ignored

In [ ]:
# orig 4096
train()

2022-01-12 20:59:25 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-01-12 20:59:29 - DEBUG:	train #1 lr = 1.00e-04 loss = 4.019592 / 0.000000 train = 0.060547 
2022-01-12 20:59:30 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.375838 / 0.000000 train = 0.130859 
2022-01-12 20:59:31 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.128880 / 0.000000 train = 0.207031 
2022-01-12 20:59:32 - DEBUG:	train #4 lr = 1.00e-04 loss = 2.895303 / 0.000000 train = 0.275391 
2022-01-12 20:59:33 - DEBUG:	train #5 lr = 1.00e-04 loss = 2.794957 / 0.000000 train = 0.298828 
2022-01-12 20:59:40 - INFO:	train #10 lr = 9.99e-05 loss = 2.488120 / 0.000000 train = 0.382031 valid = 0.499594 updated
2022-01-12 20:59:52 - INFO:	train #20 lr = 9.98e-05 loss = 2.028520 / 0.000000 train = 0.501563 valid = 0.541024 updated
2022-01-12 21:00:04 - INFO:	train #30 lr = 9.97e-05 loss = 1.718285 / 0.000000 train = 0.557813 valid = 0.589358 updated
2022-01-12 21:00:16 - INFO:	train #40 lr = 9.96e-05 loss = 1.506872 / 0.000000 trai

KeyboardInterrupt: ignored

In [ ]:
# orig, classifier only 2 layers
train()

2022-01-11 20:09:34 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-01-11 20:09:37 - DEBUG:	train #1 lr = 1.00e-04 loss = 4.042424 / 0.000000 train = 0.052734 
2022-01-11 20:09:38 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.570673 / 0.000000 train = 0.113281 
2022-01-11 20:09:39 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.352400 / 0.000000 train = 0.185547 
2022-01-11 20:09:40 - DEBUG:	train #4 lr = 1.00e-04 loss = 3.187241 / 0.000000 train = 0.222656 
2022-01-11 20:09:41 - DEBUG:	train #5 lr = 1.00e-04 loss = 2.883929 / 0.000000 train = 0.304688 
2022-01-11 20:09:47 - INFO:	train #10 lr = 9.99e-05 loss = 2.591254 / 0.000000 train = 0.356641 valid = 0.478067 updated
2022-01-11 20:09:57 - INFO:	train #20 lr = 9.98e-05 loss = 2.068822 / 0.000000 train = 0.476758 valid = 0.593420 updated
2022-01-11 20:10:08 - INFO:	train #30 lr = 9.97e-05 loss = 1.783607 / 0.000000 train = 0.548633 valid = 0.614135 updated
2022-01-11 20:10:19 - INFO:	train #40 lr = 9.96e-05 loss = 1.520811 / 0.000000 trai

KeyboardInterrupt: ignored

In [ ]:
# orig, omit upload when idim==odim, use lowrank linear
train()

2022-01-09 00:01:57 - INFO:	train epoch = 1 ~ 100000 threshold = 1.0
2022-01-09 00:02:01 - DEBUG:	train #1 lr = 1.00e-04 loss = 4.099680 train = 0.027344 
2022-01-09 00:02:02 - DEBUG:	train #2 lr = 1.00e-04 loss = 3.825120 train = 0.060547 
2022-01-09 00:02:03 - DEBUG:	train #3 lr = 1.00e-04 loss = 3.407600 train = 0.146484 
2022-01-09 00:02:03 - DEBUG:	train #4 lr = 1.00e-04 loss = 3.224407 train = 0.218750 
2022-01-09 00:02:04 - DEBUG:	train #5 lr = 1.00e-04 loss = 3.127806 train = 0.226562 
2022-01-09 00:02:09 - INFO:	train #10 lr = 9.99e-05 loss = 2.760747 train = 0.331641 valid = 0.416328 updated
2022-01-09 00:02:18 - INFO:	train #20 lr = 9.98e-05 loss = 2.364871 train = 0.423828 valid = 0.531682 updated
2022-01-09 00:02:28 - INFO:	train #30 lr = 9.97e-05 loss = 2.034938 train = 0.515820 valid = 0.577985 updated
2022-01-09 00:02:37 - INFO:	train #40 lr = 9.96e-05 loss = 1.804515 train = 0.558984 valid = 0.616166 updated
2022-01-09 00:02:46 - INFO:	train #50 lr = 9.95e-05 loss = 1.

KeyboardInterrupt: ignored

# **Test**

In [ ]:
from dataset import make_batch_train, make_batch_eval

global batch_size
num_workers = 0
batch_size = 64

for name, dataset in zip(['valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    dataset.mem = torch.load(f'{OUTPUT}/{name}_data{prefix}.pth')

mbtrain = make_batch_train
# mbtrain = make_batch_train_augment

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=mbtrain, pin_memory=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)

In [ ]:
ckpt = torch.load(f"{OUTPUT}/trained_best{prefix}.pth")
# ckpt = torch.load(f"{OUTPUT}/trained_14000.pth")
model.load_state_dict(ckpt['encoder'])
linear.load_state_dict(ckpt['linear'])
batch_size = 64

In [ ]:
evaluate(model, linear, valid_loader)

2022-01-07 11:32:58 - INFO:	loader # = 39
2022-01-07 11:32:58 - DEBUG:	test #0 correct = 0.875000
2022-01-07 11:32:58 - DEBUG:	test #1 correct = 0.875000
2022-01-07 11:32:58 - DEBUG:	test #2 correct = 0.875000
2022-01-07 11:32:58 - DEBUG:	test #3 correct = 0.882812
2022-01-07 11:32:58 - DEBUG:	test #4 correct = 0.881250
2022-01-07 11:32:58 - DEBUG:	test #5 correct = 0.877604
2022-01-07 11:32:58 - DEBUG:	test #6 correct = 0.877232
2022-01-07 11:32:59 - DEBUG:	test #7 correct = 0.869141
2022-01-07 11:32:59 - DEBUG:	test #8 correct = 0.864583
2022-01-07 11:32:59 - DEBUG:	test #9 correct = 0.867188
2022-01-07 11:32:59 - DEBUG:	test #10 correct = 0.872159
2022-01-07 11:32:59 - DEBUG:	test #11 correct = 0.876302
2022-01-07 11:32:59 - DEBUG:	test #12 correct = 0.879808
2022-01-07 11:32:59 - DEBUG:	test #13 correct = 0.878348
2022-01-07 11:32:59 - DEBUG:	test #14 correct = 0.877083
2022-01-07 11:32:59 - DEBUG:	test #15 correct = 0.878906
2022-01-07 11:32:59 - DEBUG:	test #16 correct = 0.881434

0.8862713241267263

In [ ]:
evaluate(model, linear, test_loader)

2022-01-07 11:33:02 - INFO:	loader # = 39
2022-01-07 11:33:02 - DEBUG:	test #0 correct = 0.781250
2022-01-07 11:33:02 - DEBUG:	test #1 correct = 0.789062
2022-01-07 11:33:02 - DEBUG:	test #2 correct = 0.807292
2022-01-07 11:33:02 - DEBUG:	test #3 correct = 0.839844
2022-01-07 11:33:02 - DEBUG:	test #4 correct = 0.840625
2022-01-07 11:33:02 - DEBUG:	test #5 correct = 0.841146
2022-01-07 11:33:02 - DEBUG:	test #6 correct = 0.845982
2022-01-07 11:33:02 - DEBUG:	test #7 correct = 0.857422
2022-01-07 11:33:02 - DEBUG:	test #8 correct = 0.857639
2022-01-07 11:33:02 - DEBUG:	test #9 correct = 0.859375
2022-01-07 11:33:02 - DEBUG:	test #10 correct = 0.866477
2022-01-07 11:33:02 - DEBUG:	test #11 correct = 0.869792
2022-01-07 11:33:03 - DEBUG:	test #12 correct = 0.872596
2022-01-07 11:33:03 - DEBUG:	test #13 correct = 0.869420
2022-01-07 11:33:03 - DEBUG:	test #14 correct = 0.868750
2022-01-07 11:33:03 - DEBUG:	test #15 correct = 0.866211
2022-01-07 11:33:03 - DEBUG:	test #16 correct = 0.861213

0.8476499189627229

In [ ]:
evaluate(model, linear, train_loader)

2021-11-29 22:03:37 - INFO:	loader # = 1076
2021-11-29 22:03:37 - DEBUG:	test #0 correct = 1.000000
2021-11-29 22:03:37 - DEBUG:	test #1 correct = 1.000000
2021-11-29 22:03:37 - DEBUG:	test #2 correct = 1.000000
2021-11-29 22:03:37 - DEBUG:	test #3 correct = 1.000000
2021-11-29 22:03:37 - DEBUG:	test #4 correct = 1.000000
2021-11-29 22:03:37 - DEBUG:	test #5 correct = 0.997396
2021-11-29 22:03:37 - DEBUG:	test #6 correct = 0.997768
2021-11-29 22:03:38 - DEBUG:	test #7 correct = 0.998047
2021-11-29 22:03:38 - DEBUG:	test #8 correct = 0.998264
2021-11-29 22:03:38 - DEBUG:	test #9 correct = 0.998437
2021-11-29 22:03:38 - DEBUG:	test #10 correct = 0.998580
2021-11-29 22:03:38 - DEBUG:	test #11 correct = 0.998698
2021-11-29 22:03:38 - DEBUG:	test #12 correct = 0.998798
2021-11-29 22:03:38 - DEBUG:	test #13 correct = 0.998884
2021-11-29 22:03:38 - DEBUG:	test #14 correct = 0.998958
2021-11-29 22:03:38 - DEBUG:	test #15 correct = 0.999023
2021-11-29 22:03:38 - DEBUG:	test #16 correct = 0.9990

KeyboardInterrupt: ignored

# **Scratch**

In [ ]:
for name, param in model.named_parameters():
    if 'srate' in name:
        print(name, param.item())